<a href="https://colab.research.google.com/github/Arun-purakkatt/Deep_learning_TensorFlow/blob/master/Multiclassclassification_on_Stackoverflow_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## multiclass classification on Stack Overflow questions

Exercise: multiclass classification on Stack Overflow questions
This tutorial showed how to train a binary classifier from scratch on the IMDB dataset. As an exercise, you can modify this notebook to train a multiclass classifier to predict the tag of a programming question on Stack Overflow.

We have prepared a dataset for you to use containing the body of several thousand programming questions (for example, "How can sort a dictionary by value in Python?") posted to Stack Overflow. Each of these is labeled with exactly one tag (either Python, CSharp, JavaScript, or Java). Your task is to take a question as input, and predict the appropriate tag, in this case, Python.

The dataset you will work with contains several thousand questions extracted from the much larger public Stack Overflow dataset on BigQuery, which contains more than 17 million posts.

After downloading the dataset, you will find it has a similar directory structure to the IMDB dataset you worked with previously:

train/
...python/
......0.txt
......1.txt
...javascript/
......0.txt
......1.txt
...csharp/
......0.txt
......1.txt
...java/
......0.txt
......1.txt
Note: to increase the difficulty of the classification problem, we have replaced any occurences of the words Python, CSharp, JavaScript, or Java in the programming questions with the word blank (as many questions contain the language they're about).

To complete this exercise, you should modify this notebook to work with the Stack Overflow dataset by making the following modifications:

At the top of your notebook, update the code that downloads the IMDB dataset with code to download the Stack Overflow dataset we have prepreared. As the Stack Overflow dataset has a similar directory structure, you will not need to make many modifications.

Modify the last layer of your model to read Dense(4), as there are now four output classes.

When you compile your model, change the loss to SparseCategoricalCrossentropy. This is the correct loss function to use for a multiclass classification problem, when the labels for each class are integers (in our case, they can be 0, 1, 2, or 3).

Once these changes are complete, you will be able to train a multiclass classifier.

In [2]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
print(tf.__version__)

2.3.0


In [4]:
#Setiment Analysis

# Download the data set

In [5]:
#Download dataset

url = "http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

dataset = tf.keras.utils.get_file("stack_overflow_16k.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

6053888/6053168 [==============================] - 0s 0us/step


In [11]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [12]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [13]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'test', batch_size=batch_size)

Found 8000 files belonging to 4 classes.


# Explore the data set

In [14]:

print(raw_train_ds.class_names)

['csharp', 'java', 'javascript', 'python']


In [15]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default constru

# Prepare Data for Training

In [19]:
max_features = 5000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [20]:
# Make a text-only dataset (no labels) and call adapt
text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Vectorize the data

In [21]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

## Configure the dataset for performance

In [22]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Build the model

This model has not be tuned in any way, the goal is to show you the mechanics.

In [23]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(4)])

In [24]:
model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer='adam', 
    metrics=['accuracy'])

## Train the model

In [25]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5)

Epoch 1/5
200/200 [==============================] - 9s 46ms/step - loss: 1.3732 - accuracy: 0.3100 - val_loss: 1.3554 - val_accuracy: 0.4369
Epoch 2/5
200/200 [==============================] - 7s 37ms/step - loss: 1.3200 - accuracy: 0.4589 - val_loss: 1.2791 - val_accuracy: 0.6275
Epoch 3/5
200/200 [==============================] - 7s 37ms/step - loss: 1.2204 - accuracy: 0.5816 - val_loss: 1.1627 - val_accuracy: 0.6581
Epoch 4/5
200/200 [==============================] - 7s 37ms/step - loss: 1.0992 - accuracy: 0.6578 - val_loss: 1.0436 - val_accuracy: 0.7081
Epoch 5/5
200/200 [==============================] - 7s 37ms/step - loss: 0.9847 - accuracy: 0.7005 - val_loss: 0.9395 - val_accuracy: 0.7300


# Evaluate the model

In [26]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 4s 16ms/step - loss: 0.9660 - accuracy: 0.7086
Loss:  0.9659901261329651
Accuracy:  0.7086250185966492
